In [1]:
!pip install transformers

In [2]:
from transformers import CLIPProcessor, CLIPModel

from PIL import Image # per debug
import requests # per debug

from torch import nn

In [3]:
# Pretrained CLIP loading...
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [6]:
class MyModel(nn.Module):
  def __init__(self, task_a_out, task_b_out, num_linear_layers, input_dim=1024, hidden_dim=512, drop_value=0.2): # input_dim = 1024 supposing that late fusion among embeddings is adopted
    super().__init__()
    self.head_task_a = nn.Linear(hidden_dim, task_a_out)
    self.head_task_b = nn.Linear(hidden_dim, task_b_out)

    # da implementare il preprocess per il late fusion
    self.layers = []

    # add normalization
    for i in range(num_linear_layers):
      if i == 0:
        self.layers.append(nn.Linear(input_dim, hidden_dim))
      else:
        self.layers.append(nn.Linear(hidden_dim, hidden_dim))

      self.layers.append(nn.Dropout(drop_value))
      self.layers.append(nn.ReLU())


  def forward(self, x):
    for layer in self.layers:
      x = layer(x)

    return self.head_task_a(x), self.head_task_b(x)

In [7]:
# testing the model...
import torch

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat"], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)

#print(f"text_embeds shape: {outputs['text_embeds'].shape}")
#print(f"image_embeds shape: {outputs['image_embeds'].shape}")

model_input = torch.cat([outputs['text_embeds'], outputs['image_embeds']], dim=1)
model_to_test = MyModel(2, 4, 10)

miao, miaomiao = model_to_test(model_input)

print(f"head_task_a: {miao}")
print(f"head_task_b: {miaomiao}")


head_task_a: tensor([[-0.0186, -0.0432]], grad_fn=<AddmmBackward0>)
head_task_b: tensor([[ 0.0063, -0.0361,  0.0323,  0.0150]], grad_fn=<AddmmBackward0>)
